In [ ]:
# | default_exp _components._subprocess

In [ ]:
# | export


import asyncio
import platform
import signal
from typing import *
from types import FrameType

import asyncer
import typer

from fastkafka._components.logger import get_logger

In [ ]:
import sys
import os
import platform
from time import sleep

from fastkafka._components.logger import suppress_timestamps

In [ ]:
# | notest

# allows async calls in notebooks

import nest_asyncio

In [ ]:
# | notest

nest_asyncio.apply()

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


async def terminate_asyncio_process(p: asyncio.subprocess.Process) -> None:
    """
    Terminates an asyncio process.

    Args:
        p: The asyncio.subprocess.Process instance.

    Returns:
        None.
    """
    logger.info(f"terminate_asyncio_process(): Terminating the process {p.pid}...")
    # Check if SIGINT already propagated to process
    try:
        await asyncio.wait_for(p.wait(), 1)
        logger.info(
            f"terminate_asyncio_process(): Process {p.pid} was already terminated."
        )
        return
    except asyncio.TimeoutError:
        pass

    for i in range(3):
        if platform.system() == "Windows":
            import psutil

            parent = psutil.Process(p.pid)
            children = parent.children(recursive=True)
            for child in children:
                child.kill()
        else:
            p.terminate()
        try:
            await asyncio.wait_for(p.wait(), 10)
            logger.info(f"terminate_asyncio_process(): Process {p.pid} terminated.")
            return
        except asyncio.TimeoutError:
            logger.warning(
                f"terminate_asyncio_process(): Process {p.pid} not terminated, retrying..."
            )

    logger.warning(f"Killing the process {p.pid}...")
    p.kill()
    await p.wait()
    logger.warning(f"terminate_asyncio_process(): Process {p.pid} killed!")

In [ ]:
if platform.system() == "Windows":
    code = 'import datetime; print(datetime.datetime.now())'
    proc = await asyncio.create_subprocess_exec(
        sys.executable, '-c', code,
        stdout=asyncio.subprocess.PIPE)
else:
    proc = await asyncio.create_subprocess_exec(
        "watch", "-n", "0.1", "date", stdout=asyncio.subprocess.PIPE
    )
sleep(3)
await terminate_asyncio_process(proc)
outputs, _ = await proc.communicate()

print(outputs.decode("utf-8"))

assert proc.returncode == 0, f"{command} returns {proc.returncode=}, {proc.stderr=}"

[INFO] __main__: terminate_asyncio_process(): Terminating the process 743...
[INFO] __main__: terminate_asyncio_process(): Process 743 terminated.
>?47hEvery 0.1s: datedavor-fastkafka-devel: Tue Feb  7 15:05:41 2023Tue Feb  7 15:05:41 UTC 2023223344558


In [ ]:
# | export


async def run_async_subprocesses(
    commands: List[str], commands_args: List[List[Any]], *, sleep_between: int = 0
) -> None:
    """
    Runs multiple async subprocesses.

    Args:
        commands: A list of commands to execute.
        commands_args: A list of argument lists for each command.
        sleep_between: The sleep duration in seconds between starting each subprocess.

    Returns:
        None.
    """
    loop = asyncio.get_event_loop()

    HANDLED_SIGNALS = (
        signal.SIGINT,  # Unix signal 2. Sent by Ctrl+C.
        signal.SIGTERM,  # Unix signal 15. Sent by `kill <pid>`.
    )
    if platform.system() == "Windows":
            HANDLED_SIGNALS = (*HANDLED_SIGNALS, signal.SIGBREAK)  # type: ignore

    d = {"should_exit": False}

    def handle_windows_exit(
        signum: int, frame: Optional[FrameType], d: Dict[str, bool] = d
    ) -> None:
        d["should_exit"] = True

    def handle_exit(sig: int, d: Dict[str, bool] = d) -> None:
        d["should_exit"] = True

    for sig in HANDLED_SIGNALS:
        if platform.system() == "Windows":
            signal.signal(sig, handle_windows_exit)
        else:
            loop.add_signal_handler(sig, handle_exit, sig)

    async with asyncer.create_task_group() as tg:
        tasks = []
        for cmd, args in zip(commands, commands_args):
            tasks.append(
                tg.soonify(asyncio.create_subprocess_exec)(
                    cmd,
                    *args,
                    stdout=asyncio.subprocess.PIPE,
                    stdin=asyncio.subprocess.PIPE,
                )
            )
            await asyncio.sleep(sleep_between)

    procs = [task.value for task in tasks]

    async def log_output(
        output: Optional[asyncio.StreamReader], pid: int, d: Dict[str, bool] = d
    ) -> None:
        if output is None:
            raise RuntimeError("Expected StreamReader, got None. Is stdout piped?")
        while not output.at_eof():
            outs = await output.readline()
            if outs != b"":
                typer.echo(f"[{pid:03d}]: " + outs.decode("utf-8"), nl=False)

    async with asyncer.create_task_group() as tg:
        for proc in procs:
            tg.soonify(log_output)(proc.stdout, proc.pid)

        while not d["should_exit"]:
            await asyncio.sleep(0.2)

        typer.echo("Starting process cleanup, this may take a few seconds...")
        for proc in procs:
            tg.soonify(terminate_asyncio_process)(proc)

    for proc in procs:
        output, _ = await proc.communicate()
        if output:
            typer.echo(f"[{proc.pid:03d}]: " + output.decode("utf-8"), nl=False)

    returncodes = [proc.returncode for proc in procs]
    if not returncodes == [0] * len(procs):
        typer.secho(
            f"Return codes are not all zero: {returncodes}",
            err=True,
            fg=typer.colors.RED,
        )
        raise typer.Exit(1)

In [ ]:
# | notest

# async with asyncer.create_task_group() as tg:
#     tg.soonify(run_async_subprocesses)(["watch"]*4, [["-n", "0.1", "date"]]*4, sleep_between=1)
#     await asyncio.sleep(3)

In [ ]:
# # | export


# @contextmanager
# def run_in_process(
#     target: Callable[..., Any]
# ) -> Generator[multiprocessing.Process, None, None]:
#     p = multiprocessing.Process(target=target)
#     try:
#         p.start()
#         yield p
#     except Exception as e:
#         print(f"Exception raised {e=}")
#     finally:
#         p.terminate()
#         p.join()